In [1]:
%matplotlib inline
import sys
print(sys.version)
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)
import matplotlib.pyplot as plt

3.3.2 (v3.3.2:d047928ae3f6, May 13 2013, 13:52:24) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
1.9.2
0.16.2


In this section we will be analyzing some financial data. Now pandas gives us access to some data through pandas.io.data

This is basically pandas remote data access:
http://pandas.pydata.org/pandas-docs/stable/remote_data.html

Functions from pandas.io.data and pandas.io.ga extract data from various Internet sources into a DataFrame. Currently the following sources are supported:

- Yahoo! Finance
- Google Finance
- St.Louis FED (FRED)
- Kenneth French’s data library
- World Bank
- Google Analytics

So let’s grab some  stocks from yahoo data with pandas.io.data. I've seen this list actively change so it's a good idea to see what is available to you - there's likely some really cool plugins that will continue to be built.

In [2]:
import pandas.io.data

In [3]:
?pandas.io.data # <tab>


Now there’s a lot of volatility in oil right now. It’s been rough for producers to say the least. So let’s check out some stocks that are involved in that market.

First we’ll set start and end dates these are just date times. Now I can do this with date times in python but I can also just get a datetime with pandas which can parse a string to pull out a date time. This ends up being pretty handy.

In [7]:
import datetime
print(datetime.datetime(2010,1,1))

2010-01-01 00:00:00



WTI which is W&T Offshore Inc. They drill in the gulf of Mexico.

Let’s also check out

CHK or Chesapeake Energy Corporation.

Tesla Motors

and finally CBAK which is China Bak Battery Incorporated.


In [8]:
start = pd.to_datetime('2010-1-1')
end = datetime.datetime(2015,1,1)
ticker_symbols = ['WTI','CHK','TSLA','CBAK']

In [9]:
wti = pd.io.data.get_data_yahoo(ticker_symbols[0],start=start,end=end)

In [10]:
wti.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,11.90,12.46,11.86,12.26,838800,9.824887
2010-01-05,12.30,12.63,12.17,12.34,625400,9.888997
2010-01-06,12.41,12.65,12.39,12.58,604700,10.081328
2010-01-07,12.60,12.70,12.24,12.45,565300,9.977149
2010-01-08,12.37,12.54,12.12,12.50,521100,10.017218


Now we can get these one by one in a for loop...

In [12]:
for symbol in ticker_symbols:
    print(symbol)
    df = pd.io.data.get_data_yahoo(symbol,start=start,end=end)

WTI
CHK
TSLA
CBAK


or we can just get them all by passing in the list....

In [13]:
panl = pd.io.data.get_data_yahoo(ticker_symbols,start=start,end=end)

But we get something different back that we haven’t encountered yet. This is a panel. Now panels are advanced and explaining their use case is outside of this video. However, I’ll give you the basics.


Panels are 3 dimensional containers that we can query on each of those dimensions.


In [14]:
panl

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 1258 (major_axis) x 4 (minor_axis)
Items axis: Open to Adj Close
Major_axis axis: 2010-01-04 00:00:00 to 2014-12-31 00:00:00
Minor_axis axis: CBAK to WTI


We can see they’ve got an items axis, a major axis and a minor axis. Panels are a core part of pandas however they are much less used in pandas and therefore are a bit neglected as of now. That’s not just me trying to avoid the topic - which would be my suspicion if I heard that. But that ’s stated almost verbatim in the docs.

http://pandas-docs.github.io/pandas-docs-travis/dsintro.html#from-dataframe-using-to-panel-method


In [15]:
type(panl)

pandas.core.panel.Panel


However let’s touch on the basis because you may come across them.


We've got a lot of the basic methods like shape.


In [16]:
panl.shape

(6, 1258, 4)

We’ve got these three axes so we want to query data in them. We've got to do that a bit differently.

Since we know these axis values we can query them.

Now items are done like standard DataFrame columns with dot syntax.


In [17]:
panl.Open.head()

,CBAK,CHK,TSLA,WTI
Date,,,,
2010-01-04,2.90,27.429997,NaN,11.90
2010-01-05,2.72,28.300002,NaN,12.30
2010-01-06,2.98,29.209995,NaN,12.41
2010-01-07,2.90,28.629998,NaN,12.60
2010-01-08,2.90,28.389996,NaN,12.37


The major and minor axises are done differently. with the major_xs and minor_xs commands.

In [18]:
panl.major_xs('2013-5-1')

,Open,High,Low,Close,Volume,Adj Close
CBAK,0.620000,0.790000,0.60,0.770000,56800,0.770000
CHK,19.900002,19.979996,18.86,19.190006,17267300,17.569929
TSLA,55.990002,55.990002,53.00,53.279999,2742800,53.279999
WTI,11.540000,11.540000,11.03,11.220000,720500,10.394898


In [19]:
panl.minor_xs('CHK').head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,27.429997,28.109996,26.920004,28.089999,31146800,24.503810
2010-01-05,28.300002,29.120002,28.199999,28.970004,28692700,25.271467
2010-01-06,29.209995,29.220005,28.530005,28.649996,16055000,24.992314
2010-01-07,28.629998,28.799995,28.180002,28.720002,13906600,25.053382
2010-01-08,28.389996,28.919998,28.050002,28.909998,11656400,25.219122


Some summary statistics are available to us like mean on the panel.

In [20]:
panl.mean()

,Open,High,Low,Close,Volume,Adj Close
CBAK,1.729793,1.801296,1.670469,1.726248,117253.020668,4.791367
CHK,24.194174,24.530963,23.802076,24.165184,14230891.096979,21.921955
TSLA,88.218820,89.988099,86.329102,88.177315,4136559.595070,88.177315
WTI,16.080199,16.418887,15.720723,16.069173,806965.580286,14.329300


We can perform different kinds of selections and transposition using the major and minor axes however as I said above I'm not going to cover this material. However I’m going to convert this panel to a data frame to show you how to do that and in the process and we’re going to cover a new topic!

Now when we convert the panel to a data frame with the to_frame command - we can see it looks a bit different.

In [21]:
df = panl.to_frame()
df.head()

Open       High        Low      Close    Volume  \
Date       minor                                                         
2010-01-04 CBAK    2.900000   2.900000   2.670000   2.700000    456600   
           CHK    27.429997  28.109996  26.920004  28.089999  31146800   
           WTI    11.900000  12.460000  11.860000  12.260000    838800   
2010-01-05 CBAK    2.720000   3.100000   2.690000   2.850000   1179500   
           CHK    28.300002  29.120002  28.199999  28.970004  28692700   

                  Adj Close  
Date       minor             
2010-01-04 CBAK   13.500000  
           CHK    24.503810  
           WTI     9.824887  
2010-01-05 CBAK   14.250000  
           CHK    25.271467


But when we use the head method to see the first 5 we see things are a little different and that’s because we now have multiple indices or a hierarchical or multi index. Now hierarchical indexes are extremely powerful but they’re beyond the scope of this current video - I'll touch on them a bit later in this section.

What you need to know right now is that there are levels that are stacked on one another and those can be queried.


In [22]:
df.index.levels

FrozenList([[2010-01-04 00:00:00, 2010-01-05 00:00:00, 2010-01-06 00:00:00, 2010-01-07 00:00:00, 2010-01-08 00:00:00, 2010-01-11 00:00:00, 2010-01-12 00:00:00, 2010-01-13 00:00:00, 2010-01-14 00:00:00, 2010-01-15 00:00:00, 2010-01-19 00:00:00, 2010-01-20 00:00:00, 2010-01-21 00:00:00, 2010-01-22 00:00:00, 2010-01-25 00:00:00, 2010-01-26 00:00:00, 2010-01-27 00:00:00, 2010-01-28 00:00:00, 2010-01-29 00:00:00, 2010-02-01 00:00:00, 2010-02-02 00:00:00, 2010-02-03 00:00:00, 2010-02-04 00:00:00, 2010-02-05 00:00:00, 2010-02-08 00:00:00, 2010-02-09 00:00:00, 2010-02-10 00:00:00, 2010-02-11 00:00:00, 2010-02-12 00:00:00, 2010-02-16 00:00:00, 2010-02-17 00:00:00, 2010-02-18 00:00:00, 2010-02-19 00:00:00, 2010-02-22 00:00:00, 2010-02-23 00:00:00, 2010-02-24 00:00:00, 2010-02-25 00:00:00, 2010-02-26 00:00:00, 2010-03-01 00:00:00, 2010-03-02 00:00:00, 2010-03-03 00:00:00, 2010-03-04 00:00:00, 2010-03-05 00:00:00, 2010-03-08 00:00:00, 2010-03-09 00:00:00, 2010-03-10 00:00:00, 2010-03-11 00:00:00, 

In [23]:
print(len(df.index.levels))

2



Alright we have our dataset in a hierarchical index. but that's not what we want to work with right now which will likely come up when you're analyzing data - you'll want to completely reset your index. Well have no fear, we can do that with the reset index command.

You may find yourself using this often just to get back to square one and start over when performing analysis.

In [24]:
df.reset_index()

,Date,minor,Open,High,Low,Close,Volume,Adj Close
0,2010-01-04,CBAK,2.900000,2.900000,2.670000,2.700000,456600,13.500000
1,2010-01-04,CHK,27.429997,28.109996,26.920004,28.089999,31146800,24.503810
2,2010-01-04,WTI,11.900000,12.460000,11.860000,12.260000,838800,9.824887
3,2010-01-05,CBAK,2.720000,3.100000,2.690000,2.850000,1179500,14.250000
4,2010-01-05,CHK,28.300002,29.120002,28.199999,28.970004,28692700,25.271467
5,2010-01-05,WTI,12.300000,12.630000,12.170000,12.340000,625400,9.888997
6,2010-01-06,CBAK,2.980000,3.050000,2.850000,2.880000,633500,14.400000
7,2010-01-06,CHK,29.209995,29.220005,28.530005,28.649996,16055000,24.992314
8,2010-01-06,WTI,12.410000,12.650000,12.390000,12.580000,604700,10.081328
9,2010-01-07,CBAK,2.900000,3.090000,2.780000,2.900000,784600,14.500000


In [25]:
df.reset_index(inplace=True)

Now we've reset our index. Now I don't want to work with this data set as I'm not an expert on financial data however we'll be working with a really cool data set in our next video. We'll be working with an airplane data set that has flights across the country. This is going to give us the opportunity to work on a ton of cool problems.

In [26]:
df

,Date,minor,Open,High,Low,Close,Volume,Adj Close
0,2010-01-04,CBAK,2.900000,2.900000,2.670000,2.700000,456600,13.500000
1,2010-01-04,CHK,27.429997,28.109996,26.920004,28.089999,31146800,24.503810
2,2010-01-04,WTI,11.900000,12.460000,11.860000,12.260000,838800,9.824887
3,2010-01-05,CBAK,2.720000,3.100000,2.690000,2.850000,1179500,14.250000
4,2010-01-05,CHK,28.300002,29.120002,28.199999,28.970004,28692700,25.271467
5,2010-01-05,WTI,12.300000,12.630000,12.170000,12.340000,625400,9.888997
6,2010-01-06,CBAK,2.980000,3.050000,2.850000,2.880000,633500,14.400000
7,2010-01-06,CHK,29.209995,29.220005,28.530005,28.649996,16055000,24.992314
8,2010-01-06,WTI,12.410000,12.650000,12.390000,12.580000,604700,10.081328
9,2010-01-07,CBAK,2.900000,3.090000,2.780000,2.900000,784600,14.500000
